In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random

from tqdm import tqdm

In [5]:
RND_STATE = 777

In [8]:
datasets_names = ["Тренировочный", "Валидационный", "Контрольный"]
datasets_path = ["Train_wo_na.csv", "Valid_wo_na.csv", "Control_wo_na.csv"]

train = pd.read_csv(fr'Data\{datasets_path[0]}', encoding='cp1251', index_col = 0)
valid = pd.read_csv(fr'Data\{datasets_path[1]}', encoding='cp1251', index_col = 0)
test = pd.read_csv(fr'Data\{datasets_path[2]}', encoding='cp1251', index_col = 0)

# Нормализация распределения количественных переменных

В отличие от линейной регрессии, логистическая регрессия не полагается на предположение о нормальном распределении. Однако, решение может стать более стабильным, если распределение предикторов будет многомерным нормальным.